## Data setup

### Create DuckDB table

In [1]:
import duckdb

In [6]:
import yaml
credentials_path = "../conf/base/credentials.yml"
with open(credentials_path, 'r') as f:
    credentials = yaml.safe_load(f)

In [7]:
duckdb_string = credentials["test_duckdb"]["con"]

In [2]:

a
with open()
credentials = yaml.safe_load()

con = duckdb.connect(duckdb_string)

In [3]:
import pandas as pd

In [ ]:
test_data = pd.DataFrame({
    "date": pd.date_range("2022-01-01", "2022-01-10"),
    "group": ["A","A","A","A","A","B","B","B","B","B",],
    "value":[1,2,3,4,5,6,7,8,9,10]
})

In [ ]:
con.execute('CREATE TABLE test_dataset AS SELECT * FROM test_data')

In [ ]:
con.close()

In [ ]:
import pandas as pd

### Test Ibis

In [2]:
import ibis

In [4]:
ib_con = ibis.duckdb.connect("../data/01_raw/test_db.duckdb")

In [5]:
ib_con.list_tables()

['test_dataset']

In [11]:
test_table = ib_con.table("test_dataset")

In [8]:
ib_con.con.close()

AttributeError: 'Engine' object has no attribute 'close'

In [6]:
ib_con.close()

AttributeError: 'Backend' object has no attribute 'close'

In [12]:
test_table.head(9).execute()

,date,group,value
0,2022-01-01,A,1
1,2022-01-02,A,2
2,2022-01-03,A,3
3,2022-01-04,A,4
4,2022-01-05,A,5
5,2022-01-06,B,6
6,2022-01-07,B,7
7,2022-01-08,B,8
8,2022-01-09,B,9


In [23]:
ib_con2 = ibis.connect("duckdb://data/test_db.duckdb")

In [24]:
ib_con2.list_tables()

['test_dataset']

## Kedro Dataset

### SQLAlchemy

In [30]:
from datasets.lazy_load.lazy_load_sql_dataset import LazyLoadSQLQueryDataset, Condition

In [17]:
stock_table_dataset = LazyLoadSQLQueryDataset("test_dataset", "duckdb:///data/test_db.duckdb")

In [18]:
stock_table_dataset_loader = stock_table_dataset.load()

/home/jovyan/Misc/kedro-datasets/.venv/lib/python3.8/site-packages/duckdb_engine/__init__.py:160: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [20]:
columns_to_load = ['date', 'value']
filter_conditions = [Condition('group', '=', 'A'), Condition('date', '__ge__', '2022-01-02')]

# call the loader inside a node with the selected columns and conditions
stocks = stock_table_dataset_loader(columns=columns_to_load, conditions=filter_conditions)

In [21]:
stocks

,date,value
0,2022-01-02,2
1,2022-01-03,3
2,2022-01-04,4
3,2022-01-05,5


### Ibis Kedro

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets.lazy_load.lazy_load_ibis_dataset import IbisDataSet

In [3]:
test_ibis_data = IbisDataSet("test_dataset", {"con": "duckdb://data/test_db.duckdb"})

In [5]:
ibis_kedro_test_table = test_ibis_data.load()

In [6]:
ibis_kedro_test_table.head(9).execute()

,date,group,value
0,2022-01-01,A,1
1,2022-01-02,A,2
2,2022-01-03,A,3
3,2022-01-04,A,4
4,2022-01-05,A,5
5,2022-01-06,B,6
6,2022-01-07,B,7
7,2022-01-08,B,8
8,2022-01-09,B,9
